### Load Dependencies

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Load Dataset

In [ ]:
beats = np.load("../preprocessed_beats.npy")  # shape: [N, T]
print("Loaded beats:", beats.shape)

### Visualize Sample Beats

In [ ]:
plt.figure(figsize=(10, 3))
for i in range(5):
    plt.plot(beats[i], label=f'beat {i}')
plt.title("Original ECG beats")
plt.legend()
plt.show()

### Bootstrapping